In [9]:
import numpy as np
import pandas as pd
from PIL import Image
from os import listdir
from os.path import join, basename
import struct
import pickle
import json
import os
from scipy import misc
import datetime as dt
# import matplotlib.pyplot as plt
# %matplotlib inline

In [10]:
# %pylab inline
from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.util.common import *
from bigdl.dataset.transformer import *
from bigdl.dataset import mnist
from transformer import *

In [11]:

def scala_T(input_T):
    """
    Helper function for building Inception layers. Transforms a list of numbers to a dictionary with ascending keys 
    and 0 appended to the front. Ignores dictionary inputs. 
    
    :param input_T: either list or dict
    :return: dictionary with ascending keys and 0 appended to front {0: 0, 1: realdata_1, 2: realdata_2, ...}
    """    
    if type(input_T) is list:
        # insert 0 into first index spot, such that the real data starts from index 1
        temp = [0]
        temp.extend(input_T)
        return dict(enumerate(temp))
    # if dictionary, return it back
    return input_T

In [98]:
def Inception_Layer_v1(input_size, config, name_prefix=""):
    """
    Builds the inception-v1 submodule, a local network, that is stacked in the entire architecture when building
    the full model.  
    
    :param input_size: dimensions of input coming into the local network
    :param config: ?
    :param name_prefix: string naming the layers of the particular local network
    :return: concat container object with all of the Sequential layers' ouput concatenated depthwise
    """        
    
    '''
    Concat is a container who concatenates the output of it's submodules along the provided dimension: all submodules 
    take the same inputs, and their output is concatenated.
    '''
    concat = Concat(2)
    
    """
    In the above code, we first create a container Sequential. Then add the layers into the container one by one. The 
    order of the layers in the model is same with the insertion order. 
    
    """
    conv1 = Sequential()
    
    #Adding layes to the conv1 model we jus created
    
    #SpatialConvolution is a module that applies a 2D convolution over an input image.
    conv1.add(SpatialConvolution(input_size, config[1][1], 1, 1, 1, 1).set_name(name_prefix + "1x1"))
    conv1.add(ReLU(True).set_name(name_prefix + "relu_1x1"))
    concat.add(conv1)
    
    conv3 = Sequential()
    conv3.add(SpatialConvolution(input_size, config[2][1], 1, 1, 1, 1).set_name(name_prefix + "3x3_reduce"))
    conv3.add(ReLU(True).set_name(name_prefix + "relu_3x3_reduce"))
    conv3.add(SpatialConvolution(config[2][1], config[2][2], 3, 3, 1, 1, 1, 1).set_name(name_prefix + "3x3"))
    conv3.add(ReLU(True).set_name(name_prefix + "relu_3x3"))
    concat.add(conv3)
    
    
    conv5 = Sequential()
    conv5.add(SpatialConvolution(input_size,config[3][1], 1, 1, 1, 1).set_name(name_prefix + "5x5_reduce"))
    conv5.add(ReLU(True).set_name(name_prefix + "relu_5x5_reduce"))
    conv5.add(SpatialConvolution(config[3][1], config[3][2], 5, 5, 1, 1, 2, 2).set_name(name_prefix + "5x5"))
    conv5.add(ReLU(True).set_name(name_prefix + "relu_5x5"))
    concat.add(conv5)
    
    
    pool = Sequential()
    pool.add(SpatialMaxPooling(3, 3, 1, 1, 1, 1, to_ceil=True).set_name(name_prefix + "pool"))
    pool.add(SpatialConvolution(input_size, config[4][1], 1, 1, 1, 1).set_name(name_prefix + "pool_proj"))
    pool.add(ReLU(True).set_name(name_prefix + "relu_pool_proj"))
    concat.add(pool).set_name(name_prefix + "output")
    return concat

In [99]:
def Inception_v1(class_num):
    model = Sequential()
    model.add(SpatialConvolution(3, 64, 7, 7, 2, 2, 3, 3, 1, False).set_name("conv1/7x7_s2"))
    model.add(ReLU(True).set_name("conv1/relu_7x7"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool1/3x3_s2"))
    model.add(SpatialCrossMapLRN(5, 0.0001, 0.75).set_name("pool1/norm1"))
    model.add(SpatialConvolution(64, 64, 1, 1, 1, 1).set_name("conv2/3x3_reduce"))
    model.add(ReLU(True).set_name("conv2/relu_3x3_reduce"))
    model.add(SpatialConvolution(64, 192, 3, 3, 1, 1, 1, 1).set_name("conv2/3x3"))
    model.add(ReLU(True).set_name("conv2/relu_3x3"))
    model.add(SpatialCrossMapLRN(5, 0.0001, 0.75).set_name("conv2/norm2"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool2/3x3_s2"))
    model.add(Inception_Layer_v1(192, scala_T([scala_T([64]), scala_T(
         [96, 128]), scala_T([16, 32]), scala_T([32])]), "inception_3a/"))
    model.add(Inception_Layer_v1(256, scala_T([scala_T([128]), scala_T(
         [128, 192]), scala_T([32, 96]), scala_T([64])]), "inception_3b/"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True))
    model.add(Inception_Layer_v1(480, scala_T([scala_T([192]), scala_T(
         [96, 208]), scala_T([16, 48]), scala_T([64])]), "inception_4a/"))
    model.add(Inception_Layer_v1(512, scala_T([scala_T([160]), scala_T(
         [112, 224]), scala_T([24, 64]), scala_T([64])]), "inception_4b/"))
    model.add(Inception_Layer_v1(512, scala_T([scala_T([128]), scala_T(
         [128, 256]), scala_T([24, 64]), scala_T([64])]), "inception_4c/"))
    model.add(Inception_Layer_v1(512, scala_T([scala_T([112]), scala_T(
         [144, 288]), scala_T([32, 64]), scala_T([64])]), "inception_4d/"))
    model.add(Inception_Layer_v1(528, scala_T([scala_T([256]), scala_T(
         [160, 320]), scala_T([32, 128]), scala_T([128])]), "inception_4e/"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True))
    model.add(Inception_Layer_v1(832, scala_T([scala_T([256]), scala_T(
         [160, 320]), scala_T([32, 128]), scala_T([128])]), "inception_5a/"))
    model.add(Inception_Layer_v1(832, scala_T([scala_T([384]), scala_T(
         [192, 384]), scala_T([48, 128]), scala_T([128])]), "inception_5b/"))
    model.add(SpatialAveragePooling(7, 7, 1, 1).set_name("pool5/7x7_s1"))
    model.add(Dropout(0.4).set_name("pool5/drop_7x7_s1"))
    model.add(View([1024], num_input_dims=3))
    model.add(Linear(1024, class_num).set_name("loss3/classifier"))
    model.add(LogSoftMax().set_name("loss3/loss3"))
    model.reset()
    return model

## Download the images from Amazon s3

Make sure you have AWS command line interface to recursively download all images in s3 folder. You can set up aws cli from this link: http://docs.aws.amazon.com/cli/latest/userguide/cli-chap-welcome.html

In [100]:
import urllib
from os import path
DATA_ROOT = "./sample_images"
local_folder = DATA_ROOT + '/vegnonveg-fewsamples'
checkpoint_path = path.join(DATA_ROOT, "checkpoints")
IMAGE_SIZE = 224

if not path.isdir(local_folder):
    os.system('aws s3 cp --recursive s3://vegnonveg/vegnonveg-fewsamples %s' % local_folder)


## Preprocess images and save them with their Labels

I have saved the final dataset in a pickle file on my local machine, it can be saved into s3 as well

In [94]:
labels_csv = pd.read_csv(DATA_ROOT + '/vegnonveg-samples_labels.csv')
unique_labels = labels_csv['item_name'].unique().tolist()
label_dict = dict(zip(unique_labels, range(0,len(unique_labels))))
class_num = len(label_dict)

In [79]:
def crop_image(img):
    new_dim = min(img.width, img.height)
    x = (img.width - new_dim) / 2
    y = (img.height - new_dim) / 2
    cropped = img.crop((x, y, x + new_dim, y + new_dim))
    return cropped

In [80]:
transform_input = Transformer([ChannelNormalizer(0.485, 0.456, 0.406, 0.229, 0.224, 0.225),
                               TransposeToTensor(False)])


In [81]:
def find_label(labels_csv, label_dict, file_name):
    labels = labels_csv.set_index(['obs_uid'])
    str_label = labels.loc[file_name]['item_name']
    return label_dict[str_label]

In [82]:
def convert_to_np(image_path, IMAGE_SIZE):
    input_img = Image.open(image_path)
    cropped = crop_image(input_img)
    resized = cropped.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
    rgb_image = resized.convert('RGB')
    img_np = np.array(rgb_image)
    return img_np

In [83]:
'''
Save the data as a dictionary of images and their labels, save into a pickle file
'''
IMAGE_SIZE = 224
data = {}
data['images'] = []
data['labels'] = []

files = sorted(os.listdir(local_folder))
for file_name in files:
    label = find_label(labels_csv, label_dict, file_name)
    image_path = path.join(local_folder, file_name)
    img_np = convert_to_np(image_path, IMAGE_SIZE)
    data['images'].append(img_np)
    data['labels'].append(label)

# option to save this file into s3 bucket
pickle.dump(data, open(DATA_ROOT + "/processed_imgs_with_labels.pkl", 'wb'))

## Train and Test Classifiers

In [84]:
import pickle
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [85]:
data = pickle.load(open(DATA_ROOT + "/processed_imgs_with_labels.pkl", 'rb'))

## Use Stratified Train/Test Split
To make sure we have the same distribution of samples across labels in both train and test sets.

In [88]:

x_train, x_test, train_labels, test_labels = \
    train_test_split(data['images'], 
                     data['labels'], 
                     test_size=0.2, 
                     random_state=101)
                     stratify=data['labels'])
len(x_train), len(train_labels), len(x_test), len(test_labels)

(8, 8, 2, 2)

In [89]:
_, train_counts = np.unique(np.array(train_labels), return_counts=True)
train_counts = train_counts.astype(np.float) / len(train_labels)

In [91]:
# _, test_counts = np.unique(np.array(test_labels), return_counts=True)
# test_counts = test_counts.astype(np.float) / len(test_labels)
# # Difference in labels counts, %
# (train_counts - test_counts) / train_counts * 100

In [118]:

def get_rdd_sample(images, labels):
    """
    Serializes a set of images and labels for bigdl
    
    :param images: a list of images represented as numpy arrays
    :param labels: a list of labels (strings) corresponding to those images 
    :return: the final dataset serialized for bigdl
    """ 
    imgs_rdd = sc.parallelize(images)
    labels_rdd = sc.parallelize(labels)
    sample_rdd = imgs_rdd.zip(labels_rdd).map(lambda(img, label): Sample.from_ndarray(transform_input(img), np.array(label+1)))
    return sample_rdd



In [119]:
train_rdd = get_rdd_sample(x_train, train_labels)
test_rdd = get_rdd_sample(x_test, test_labels)

## Define Model

In [120]:
#intializa bigdl
init_engine()

In [129]:
# Parameters
learning_rate = 0.2
# parameters for 
batch_size = 6400 #depends on dataset
no_epochs = 2 #stop when validation accuracy doesn't improve anymore

# Network Parameters
n_classes = len(label_dict)# item_name categories


In [122]:
model = Inception_v1(n_classes)

creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialCrossMapLRN
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialCrossMapLRN
creating: createSpatialMaxPooling
creating: createConcat
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createReLU
creating: createConcat
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialConvolutio

In [123]:
optimizer = Optimizer(
    model=model,
    training_rdd=train_rdd,
    criterion=ClassNLLCriterion(),
    optim_method=SGD(learningrate=learning_rate),
    end_trigger=MaxEpoch(no_epochs),
    batch_size=batch_size)
# Set the validation logic
optimizer.set_validation(
    batch_size=batch_size,
    val_rdd=test_rdd,
    trigger=EveryEpoch(),
    val_method=[Top1Accuracy()]
)

app_name= 'vegnonveg' # + dt.datetime.now().strftime("%Y%m%d-%H%M%S")
train_summary = TrainSummary(log_dir='/tmp/bigdl_summaries',
                                     app_name=app_name)
train_summary.set_summary_trigger("Parameters", SeveralIteration(50))
val_summary = ValidationSummary(log_dir='/tmp/bigdl_summaries',
                                        app_name=app_name)
optimizer.set_train_summary(train_summary)
optimizer.set_val_summary(val_summary)
print "saving logs to ",app_name

creating: createClassNLLCriterion
creating: createDefault
creating: createSGD
creating: createMaxEpoch
creating: createOptimizer
creating: createEveryEpoch
creating: createTop1Accuracy
creating: createTrainSummary
creating: createSeveralIteration
creating: createValidationSummary
saving logs to  vegnonveg


To visualize the training process,  you can run: 
```
tensorboard --logdir=/tmp/bigdl_summaries
```

In [124]:
# Start to train
trained_model = optimizer.optimize()
print "Optimization Done."

Optimization Done.


In [125]:
def map_predict_label(l):
    return np.array(l).argmax()
def map_groundtruth_label(l):
    return l[0] - 1
def map_to_label(l):
    return label_dict.keys()[label_dict.values().index(l)]

In [128]:
'''
Look at some predictions and their accuracy
'''
predictions = trained_model.predict(test_rdd)

num_preds = 2
truth = test_rdd.take(num_preds)
preds = predictions.take(num_preds)

for idx in range(num_preds):
    true_label = str(map_to_label(map_groundtruth_label(truth[idx].label)))
    pred_label = str(map_to_label(map_predict_label(preds[idx])))
    print idx + 1, ')', 'Ground Truth label: ', true_label
    print idx + 1, ')', 'Predicted label: ', pred_label
    print "correct" if true_label == pred_label else "wrong"

1 ) Ground Truth label:  Fresh oranges
1 ) Predicted label:  Fresh carrots
wrong
2 ) Ground Truth label:  Fresh onions
2 ) Predicted label:  Fresh carrots
wrong


In [127]:
'''
Measure Test Accuracy w/Test Set
'''
results = trained_model.test(test_rdd, len(x_test), [Top1Accuracy()])
print(results[0])

creating: createTop1Accuracy
Test result: 0.0, total_num: 2, method: Top1Accuracy
